In [16]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data (1).csv


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
data=pd.read_csv('data.csv')

In [19]:
data.head()

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Blondie,movie,"Comedy, Family",1938.0,tt0029927,6.9,886.0,"US, ZA"
1,Ariel,movie,"Comedy, Crime, Romance",1988.0,tt0094675,7.4,8752.0,JP
2,Four Rooms,movie,Comedy,1995.0,tt0113101,6.7,112543.0,"AT, DE"
3,Judgment Night,movie,"Action, Crime, Drama",1993.0,tt0107286,6.6,19282.0,US
4,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2315370.0,"AD, AT, CU, DE, IN, JP, SN"


In [20]:
# Remove specified columns
data = data.drop(['releaseYear', 'availableCountries', 'imdbId','imdbNumVotes','imdbAverageRating'], axis=1, errors='ignore')
data.head()

,title,type,genres
0,Blondie,movie,"Comedy, Family"
1,Ariel,movie,"Comedy, Crime, Romance"
2,Four Rooms,movie,Comedy
3,Judgment Night,movie,"Action, Crime, Drama"
4,Forrest Gump,movie,"Drama, Romance"


In [21]:
data.isnull().sum()

,0
title,1441
type,0
genres,1093


In [23]:
data = data.dropna()
data.isnull().sum()

,0
title,0
type,0
genres,0


In [25]:
data.shape

(49930, 3)

In [26]:
data['label'] = data['type'] + ' ' + data['genres']
data = data.drop(['type', 'genres'], axis=1)
data.head()

,title,label
0,Blondie,"movie Comedy, Family"
1,Ariel,"movie Comedy, Crime, Romance"
2,Four Rooms,movie Comedy
3,Judgment Night,"movie Action, Crime, Drama"
4,Forrest Gump,"movie Drama, Romance"


In [27]:
data.shape

(49930, 2)

In [28]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    if isinstance(text, str):  # Check if the input is a string
        words = word_tokenize(text.lower())
        words = [w for w in words if not w in stop_words and w.isalnum()]
        words = [stemmer.stem(w) for w in words]
        return ' '.join(words)
    else:
        return '' # Return an empty string for non-string inputs


data['label'] = data['label'].apply(preprocess_text)
data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,title,label
0,Blondie,movi comedi famili
1,Ariel,movi comedi crime romanc
2,Four Rooms,movi comedi
3,Judgment Night,movi action crime drama
4,Forrest Gump,movi drama romanc


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['label'])

In [30]:
data.head()

,title,label
0,Blondie,movi comedi famili
1,Ariel,movi comedi crime romanc
2,Four Rooms,movi comedi
3,Judgment Night,movi action crime drama
4,Forrest Gump,movi drama romanc


In [31]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

indices = pd.Series(data.index, index=data['title']).drop_duplicates()

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11] # Get top 10 similar movies
    movie_indices = [i[0] for i in sim_scores]
    return data['title'].iloc[movie_indices]


In [32]:
# Example usage:
movie_title = input("Enter The Full Movie Name: ")
recommendations = get_recommendations(movie_title)
print(recommendations.head())

Enter The Full Movie Name: Batman
31                  Raiders of the Lost Ark
32     Indiana Jones and the Temple of Doom
34       Indiana Jones and the Last Crusade
36                               Armageddon
100                 Star Trek: Insurrection
Name: title, dtype: object
